In [1]:
import os
os.chdir('/home/fsboedij/Data/MouseAtlas')
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)
import scanpy as sc
import copy
import torch
import scarches as sca
from scarches.dataset.trvae.data_handling import remove_sparsity
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import gdown
import sankey
import anndata
import pandas as pd
import pandas as pd
from pySankey.sankey import sankey
from matplotlib.pyplot import rc_context
import seaborn as sns
plt.rcParams['figure.figsize']=(8,8) #rescale figures
sc.settings.verbosity = 3
sc.settings.set_figure_params(dpi=200, frameon=False)
sc.set_figure_params(dpi=200)
sc.set_figure_params(figsize=(5, 5))
torch.set_printoptions(precision=10, sci_mode=False, edgeitems=12)
pd.options.mode.chained_assignment = None
import pickle

INFO:lightning_fabric.utilities.seed:Global seed set to 0
2025-01-23 09:54:49.137418: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
 captum (see https://github.com/pytorch/captum).


Preprocessing of the filtered and concatenated adata followed with extraction of the highly variable genes

In [2]:
def hvg_batch_cellranger(
    adata, 
    batch_key=None, 
    target_genes=2000, 
    flavor='cell_ranger', 
    n_bins=20, 
    adataOut=False
    ):
    """
    Method to select HVGs based on mean dispersions of genes that are highly 
    variable genes in all batches. Using a the top target_genes per batch by
    average normalize dispersion. If target genes still hasn't been reached, 
    then HVGs in all but one batches are used to fill up. This is continued 
    until HVGs in a single batch are considered.
    """
    
    #checkAdata(adata)
    #if batch_key is not None:
     #   checkBatch(batch_key, adata.obs)
    
    adata_hvg = adata if adataOut else adata.copy()

    n_batches = len(adata_hvg.obs[batch_key].cat.categories)

    # Calculate double target genes per dataset
    sc.pp.highly_variable_genes(adata_hvg,
                                flavor=flavor, 
                                n_top_genes=target_genes,
                                n_bins=n_bins, 
                                batch_key=batch_key)

    nbatch1_dispersions = adata_hvg.var['dispersions_norm'][adata_hvg.var.highly_variable_nbatches >
                                                           len(adata_hvg.obs[batch_key].cat.categories)-1]
    
    nbatch1_dispersions.sort_values(ascending=False, inplace=True)

    if len(nbatch1_dispersions) > target_genes:
        hvg = nbatch1_dispersions.index[:target_genes]
    
    else:
        enough = False
        print(f'Using {len(nbatch1_dispersions)} HVGs from full intersect set')
        hvg = nbatch1_dispersions.index[:]
        not_n_batches = 1
        
        while not enough:
            target_genes_diff = target_genes - len(hvg)

            tmp_dispersions = adata_hvg.var['dispersions_norm'][adata_hvg.var.highly_variable_nbatches ==
                                                                (n_batches-not_n_batches)]

            if len(tmp_dispersions) < target_genes_diff:
                print(f'Using {len(tmp_dispersions)} HVGs from n_batch-{not_n_batches} set')
                hvg = hvg.append(tmp_dispersions.index)
                not_n_batches += 1

            else:
                print(f'Using {target_genes_diff} HVGs from n_batch-{not_n_batches} set')
                tmp_dispersions.sort_values(ascending=False, inplace=True)
                hvg = hvg.append(tmp_dispersions.index[:target_genes_diff])
                enough=True

    print(f'Using {len(hvg)} HVGs')

    if not adataOut:
        del adata_hvg
        return hvg.tolist()
    else:
        return adata_hvg[:,hvg].copy

Specify the directory where each filtered anndata object for each study is stored in one folder

In [3]:
directory='/home/fsboedij/Data/MouseAtlas/Data/Individual_studies_final/postQC/'

Initialize all studies. Each object was loaded one at a time to accurately harmonise the meta data (i.e., Group and Control samples).

In [4]:
covid = sc.read(directory + "Mouse_Covid.h5ad")
copd_covid = sc.read(directory + "Mouse_Copd_Covid.h5ad")
copd = sc.read(directory + "Mouse_Copd.h5ad")
age = sc.read(directory + "Mouse_Age.h5ad")
cssc = sc.read(directory + "Mouse_COPD_SHAM_SARSCov2.h5ad")
herpes = sc.read(directory + "Mouse_Herpesvirus.h5ad")
hyper = sc.read(directory + "Mouse_Hyperoxia.h5ad")
tb = sc.read(directory + "Mouse_TB.h5ad")
cancer = sc.read(directory + "Mouse_Cancer.h5ad")
hypo = sc.read(directory + "Mouse_Hypoxia.h5ad")
rad = sc.read(directory + "Mouse_Radiation.h5ad")
sendai = sc.read(directory + "Mouse_Post_Sendai.h5ad")
tumor = sc.read(directory + "Mouse_TumorKO.h5ad")
fibro = sc.read(directory + "Mouse_fibrosis.h5ad")
asthma = sc.read(directory + "Mouse_asthma.h5ad")
fibage = sc.read(directory + "Mouse_Fibro_Age.h5ad")
influ = sc.read(directory + "Mouse_influenza.h5ad")

print(covid)

print('All objects are loaded')

AnnData object with n_obs × n_vars = 37280 × 34493
    obs: 'nCount_covid', 'nFeature_covid', 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'percent.mt', 'CellStatus', 'pANN_0.25_0.09_241', 'DF.classifications_0.25_0.09_241', 'DoubletStatus', 'pANN_0.25_0.09_520', 'DF.classifications_0.25_0.09_520', 'pANN_0.25_0.09_148', 'DF.classifications_0.25_0.09_148', 'pANN_0.25_0.09_368', 'DF.classifications_0.25_0.09_368', 'pANN_0.25_0.09_1110', 'DF.classifications_0.25_0.09_1110', 'pANN_0.25_0.09_33', 'DF.classifications_0.25_0.09_33', 'pANN_0.25_0.09_1015', 'DF.classifications_0.25_0.09_1015', 'nGene', 'nUMI', 'Group', 'res.0', 'res.0.1', 'res.0.2', 'res.0.3', 'res.0.4', 'res.0.5', 'res.0.6', 'res.0.7', 'res.0.8', 'res.0.9', 'res.1', 'seurat_clusters', 'infection', 'Sample.Gender', 'percent.hb', 'percent.rp', 'cluster_labels_res.0.4', 'predicted.classification_Angelidis.score', 'predicted.classification_Angelidis', 'mouseRNA.labels', 'immgen.labels', 'RNA_snn_res.0.3', 'RNA_snn_res.0.4', 'RNA_sn

Define groups from each study

In [38]:
covid.X.data = covid.raw.X.data
copd.X.data = copd.raw.X.data
copd_covid.X.data = copd_covid.raw.X.data
age.X.data = age.raw.X.data
cssc.X.data = cssc.raw.X.data
herpes.X.data = herpes.raw.X.data
hyper.X.data = hyper.raw.X.data
tb.X.data = tb.raw.X.data
hypo.X.data = hypo.raw.X.data
cancer.X.data = cancer.raw.X.data
rad.X.data = rad.raw.X.data
sendai.X.data = sendai.raw.X.data
tumor.X.data = tumor.raw.X.data
fibro.X.data = fibro.raw.X.data
asthma.X.data = asthma.raw.X.data
fibage.X.data = fibage.raw.X.data
influ.X.data = influ.raw.X.data

copd.obs['Group'] = copd.obs['Timepoint']
age.obs['Group'] = age.obs['grouping']
herpes.obs['Group'] = herpes.obs['treatment']
hyper.obs['Group'] = hyper.obs['treatment']
tb.obs['Group'] = tb.obs['treatment']
cancer.obs['Group'] = cancer.obs['Treatment']
hypo.obs['Group'] = hypo.obs['treatment']
rad.obs['Group'] = rad.obs['treatment']
sendai.obs['Group'] = sendai.obs['Treatment']
tumor.obs['Group'] = tumor.obs['treatment']
fibro.obs['Group'] = fibro.obs['treatment']
asthma.obs['Group'] = asthma.obs['SampleIDs']
influ.obs['Group'] = influ.obs['treatment']

Ensure all metadata is harmonised with all sets before proceeding

Split each study object using their 'Group' metadata into Control and 'Disease' samples

Save control and disease as separate anndata objects. To be used for the all gene objects.

Once saved find the top 2000 highly variable genes that occur in all datasets, using the top 6000 highly variable from each set, and cross correlating with each set to find the 2000

In [5]:
#separate healthy and disease samples from each study into one list to be processed separately
healthy=[
covid[(covid.obs['Group'] == 'FemaleControl')|\
	(covid.obs['Group'] == 'MaleControl')],
copd_covid[(copd_covid.obs['Group'] == 'AirControl')],
copd[(copd.obs['Group'] == '12 Week Air')],
age[(age.obs['Group'] == '3m')],
cssc[(cssc.obs['Group'] == 'AirControl')],
herpes[(herpes.obs['Group'] == 'Untreated')],
hyper[(hyper.obs['Group'] == 'Room Air')],
tb[(tb.obs['Group'] == 'Uninfected')],
cancer[(cancer.obs['Group'] == 'Control')],
hypo[(hypo.obs['Group'] == 'Room Air')],
rad[(rad.obs['Group'] == 'Control')],
sendai[(sendai.obs['Group'] == 'PBS')],
tumor[(tumor.obs['Group'] == 'Control')],
fibro[(fibro.obs['Group'] == 'Control')],
asthma[(asthma.obs['Group'] == 'Sal/SPG')],
fibage[(fibage.obs['Group'] == 'Old_Control')|\
    (fibage.obs['Group']=='Young_Control')],
influ[(influ.obs['Group'] == 'Control')],
]

disease=[
covid[(covid.obs['Group'] != 'FemaleControl')&\
	(covid.obs['Group'] != 'MaleControl')],
copd_covid[(copd_covid.obs['Group'] != 'AirControl')],
copd[(copd.obs['Group'] != '12 Week Air')],
age[(age.obs['Group'] != '3m')],
cssc[(cssc.obs['Group'] != 'AirControl')],
herpes[(herpes.obs['Group'] != 'Untreated')],
hyper[(hyper.obs['Group'] != 'Room Air')],
tb[(tb.obs['Group'] != 'Uninfected')],
cancer[(cancer.obs['Group'] != 'Control')],
hypo[(hypo.obs['Group'] != 'Room Air')],
rad[(rad.obs['Group'] != 'Control')],
sendai[(sendai.obs['Group'] != 'PBS')],
tumor[(tumor.obs['Group'] != 'Control')],
fibro[(fibro.obs['Group'] != 'Control')],
asthma[(asthma.obs['Group'] != 'Sal/SPG')],
fibage[(fibage.obs['Group'] != 'Old_Control')&\
    (fibage.obs['Group']!='Young_Control')],
influ[(influ.obs['Group'] != 'Control')],
]

adata_lists = [healthy,disease]

In [11]:
import pickle
with open('Data/step2_withloop_adatalist_healthyndisease_709.pkl', 'wb') as file:
    pickle.dump(adata_lists, file)

print('adata_list saved')

adata_list saved


In [29]:
import pickle
with open("Data/step2_withloop_adatalist_healthyndisease_2709.pkl", 'rb') as file:
    adata_lists = pickle.load(file)

In [6]:
#Combine each study into one adata containing all studies and all genes
for i in range(0,1):
    lists=adata_lists[i]
    adata=lists[0].concatenate(
    lists[1:],  # Remaining AnnData objects
    index_unique=None,  # Keep cell indices unique
    join='outer',  # Include all genes (fill missing with fill_value)
    fill_value=0  # Fill missing values with 0
    )
    print(adata)
    for col in adata.obs.columns:
        adata.obs[col] = adata.obs[col].astype(str)
        
    for col in adata.var.columns:
        adata.var[col] = adata.var[col].astype(str)

    if i==0:
        healthy=adata
        adata.obs['healthy_or_disease']='healthy'
        #adata.write("Data/full_adata_preintegration_healthy_2709.h5ad")
        print('Healthy Pre-Integration Full Adata Saved')
    elif i==1:
        disease=adata
        adata.obs['healthy_or_disease']='disease'
        #adata.write("Data/full_adata_preintegration_disease_2709.h5ad")
        print('Disease Pre-Integration Full Adata Saved')

AnnData object with n_obs × n_vars = 244836 × 44793
    obs: 'nCount_covid', 'nFeature_covid', 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'percent.mt', 'CellStatus', 'pANN_0.25_0.09_241', 'DF.classifications_0.25_0.09_241', 'DoubletStatus', 'pANN_0.25_0.09_520', 'DF.classifications_0.25_0.09_520', 'pANN_0.25_0.09_148', 'DF.classifications_0.25_0.09_148', 'pANN_0.25_0.09_368', 'DF.classifications_0.25_0.09_368', 'pANN_0.25_0.09_1110', 'DF.classifications_0.25_0.09_1110', 'pANN_0.25_0.09_33', 'DF.classifications_0.25_0.09_33', 'pANN_0.25_0.09_1015', 'DF.classifications_0.25_0.09_1015', 'nGene', 'nUMI', 'Group', 'res.0', 'res.0.1', 'res.0.2', 'res.0.3', 'res.0.4', 'res.0.5', 'res.0.6', 'res.0.7', 'res.0.8', 'res.0.9', 'res.1', 'seurat_clusters', 'infection', 'Sample.Gender', 'percent.hb', 'percent.rp', 'cluster_labels_res.0.4', 'predicted.classification_Angelidis.score', 'predicted.classification_Angelidis', 'mouseRNA.labels', 'immgen.labels', 'RNA_snn_res.0.3', 'RNA_snn_res.0.4', 'RNA_s

/tmp/ipykernel_2430724/2113810432.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  adata.obs['healthy_or_disease']='healthy'


In [7]:
healthy.

AnnData object with n_obs × n_vars = 244836 × 44793
    obs: 'nCount_covid', 'nFeature_covid', 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'percent.mt', 'CellStatus', 'pANN_0.25_0.09_241', 'DF.classifications_0.25_0.09_241', 'DoubletStatus', 'pANN_0.25_0.09_520', 'DF.classifications_0.25_0.09_520', 'pANN_0.25_0.09_148', 'DF.classifications_0.25_0.09_148', 'pANN_0.25_0.09_368', 'DF.classifications_0.25_0.09_368', 'pANN_0.25_0.09_1110', 'DF.classifications_0.25_0.09_1110', 'pANN_0.25_0.09_33', 'DF.classifications_0.25_0.09_33', 'pANN_0.25_0.09_1015', 'DF.classifications_0.25_0.09_1015', 'nGene', 'nUMI', 'Group', 'res.0', 'res.0.1', 'res.0.2', 'res.0.3', 'res.0.4', 'res.0.5', 'res.0.6', 'res.0.7', 'res.0.8', 'res.0.9', 'res.1', 'seurat_clusters', 'infection', 'Sample.Gender', 'percent.hb', 'percent.rp', 'cluster_labels_res.0.4', 'predicted.classification_Angelidis.score', 'predicted.classification_Angelidis', 'mouseRNA.labels', 'immgen.labels', 'RNA_snn_res.0.3', 'RNA_snn_res.0.4', 'RNA_s

In [8]:
def check_non_integer_values(adatas):
    # Loop through each adata object in the list
    for idx, adata in enumerate(adatas):
        # Assuming adata is an AnnData object
        if not isinstance(adata, anndata.AnnData):
            raise ValueError(f"Input at index {idx} is not an AnnData object.")
        study = str(adata.obs['study'].unique()[0])
        non_integer_values = np.any(adata.X.data % 1 != 0)
        # Print information about non-integer values
        if non_integer_values:
            print(f"Study: {study} - Non-integer values found.")
            # Add additional information or actions as needed
        else:
            print(f"Study: {study} - All values are integers.")
            adata.layers['raw'] = adata.X

for i in range(0, len(adata_lists)):
    check_non_integer_values(adata_lists[i])

Study: Mouse_Covid - All values are integers.
Study: Mouse_Copd_Covid - All values are integers.
Study: Mouse_Copd - All values are integers.
Study: Mouse_Age - All values are integers.
Study: Mouse_COPD_SHAM_SARSCov2 - All values are integers.
Study: Mouse_Herpesvirus - All values are integers.
Study: Mouse_Hyperoxia - All values are integers.
Study: Mouse_TB - All values are integers.
Study: Mouse_Cancer - All values are integers.
Study: Mouse_Hypoxia - All values are integers.
Study: Mouse_Radiation - All values are integers.
Study: Mouse_Post_Sendai - All values are integers.
Study: Mouse_TumorKO - All values are integers.
Study: Mouse_fibrosis - All values are integers.
Study: Mouse_asthma - All values are integers.
Study: Mouse_Fibro_Age - All values are integers.
Study: Mouse_influenza - All values are integers.
Study: Mouse_Covid - All values are integers.
Study: Mouse_Copd_Covid - All values are integers.
Study: Mouse_Copd - All values are integers.
Study: Mouse_Age - All valu

In [32]:
#Run only in healthy subset
sc.pp.normalize_total(healthy,target_sum=1e4)
sc.pp.log1p(healthy)
healthy.obs.study=healthy.obs.study.astype('category')
hvg=hvg_batch_cellranger(healthy,batch_key='study')
hvg=pd.DataFrame(hvg,columns=['Genes'])
#hvg.to_csv('Data/others/2kgenes_hlcamethod_2709_cellranger_healthy.csv')


normalizing counts per cell
    finished (0:00:01)
If you pass `n_top_genes`, all cutoffs are ignored.
extracting highly variable genes
    finished (0:00:23)
--> added
    'highly_variable', boolean vector (adata.var)
    'means', float vector (adata.var)
    'dispersions', float vector (adata.var)
    'dispersions_norm', float vector (adata.var)
Using 41 HVGs from full intersect set
Using 91 HVGs from n_batch-1 set
Using 141 HVGs from n_batch-2 set
Using 146 HVGs from n_batch-3 set
Using 134 HVGs from n_batch-4 set
Using 132 HVGs from n_batch-5 set
Using 139 HVGs from n_batch-6 set
Using 147 HVGs from n_batch-7 set
Using 170 HVGs from n_batch-8 set
Using 213 HVGs from n_batch-9 set
Using 225 HVGs from n_batch-10 set
Using 259 HVGs from n_batch-11 set
Using 162 HVGs from n_batch-12 set
Using 2000 HVGs


In [33]:
hvg_cellranger_old=pd.read_csv('Data/others/2kgenes_hlcamethod_2709_cellranger_healthy.csv')
print(len(set(hvg.iloc[:,0]).intersection(set(hvg_cellranger_old.iloc[:,1]))))

853


In [96]:
hvg=pd.read_csv('Data/others/2kgenes_hlcamethod_2709_cellranger_healthy.csv')
twok=hvg.iloc[:,1]

In [97]:
twok

0        S100a9
1          Gzma
2        Retnlg
3          Ccl5
4         Cxcl2
         ...   
1995       Pcp4
1996    Camk2n1
1997      Nxpe3
1998      Ffar1
1999     Cep290
Name: Genes, Length: 2000, dtype: object

Using the newly found HVG, we now add the HVG to each dataset

In [98]:
for i in range(0, len(adata_lists)):
    lists = adata_lists[i]
    len(lists) 

    for j in range(0, len(lists)):
        genes = lists[j].var.index[lists[j].var.index.isin(twok)].tolist()
        adata_sub = lists[j][:, genes].copy()
        genes_to_add = set(twok).difference(set(adata_sub.var_names))
        genes_to_add = list(genes_to_add)
        
        if genes_to_add != []:
            df_padding = pd.DataFrame(
                data=np.zeros((adata_sub.shape[0], len(genes_to_add))),
                index=adata_sub.obs_names,
                columns=genes_to_add
            )
            adata_padding = sc.AnnData(df_padding)
            adata_sub = anndata.concat([adata_sub, adata_padding], axis=1, index_unique=None, merge='unique')
        
        lists[j] = adata_sub[:, twok].copy()
        
        for col in lists[j].obs.columns:
            if pd.api.types.is_object_dtype(lists[j].obs[col]):
                lists[j].obs[col] = lists[j].obs[col].astype(str)
                
        try:
            lists[j].__dict__['_raw'].__dict__['_var'] = lists[j].__dict__['_raw'].__dict__['_var'].rename(columns={'_index': 'features'})
        except:
            pass

print('Done')

Done


Saved for integration

In [99]:
for i in range(0, len(adata_lists)):
    lists = adata_lists[i]
    len(lists)

    if i==0:
        print('Healthy adata list')
    elif i==1:
        print('Disease adata list')
    for idx, adata_obj in enumerate(lists):
         print(f"Anndata object {idx + 1}:")
         print(f"Number of observations (cells): {adata_obj.n_obs}")
         print(f"Number of variables (genes/features): {adata_obj.n_vars}")
         print(f"Number of layers: {len(adata_obj.layers)}")
         print("\n")


Healthy adata list
Anndata object 1:
Number of observations (cells): 8466
Number of variables (genes/features): 2000
Number of layers: 0


Anndata object 2:
Number of observations (cells): 6641
Number of variables (genes/features): 2000
Number of layers: 0


Anndata object 3:
Number of observations (cells): 9459
Number of variables (genes/features): 2000
Number of layers: 0


Anndata object 4:
Number of observations (cells): 7526
Number of variables (genes/features): 2000
Number of layers: 0


Anndata object 5:
Number of observations (cells): 8456
Number of variables (genes/features): 2000
Number of layers: 0


Anndata object 6:
Number of observations (cells): 5510
Number of variables (genes/features): 2000
Number of layers: 0


Anndata object 7:
Number of observations (cells): 25457
Number of variables (genes/features): 2000
Number of layers: 0


Anndata object 8:
Number of observations (cells): 7482
Number of variables (genes/features): 2000
Number of layers: 0


Anndata object 9:
Nu

In [100]:
for i in range(0,len(adata_lists)):
    lists=adata_lists[i]
    print(len(lists))
    
    for j in range(0, len(lists)):
        obj=lists[j]
        study_value = obj.obs['study'].iloc[0]
        variable_name = str(study_value)
    
        if i==0:
            print('Healthy '+variable_name)
            obj.write(f"Data/Individual_studies_final/2k/hlcamethod_cellranger_healthy2k/{variable_name}_2k_h.h5ad")
        elif i==1:
            print('Disease '+variable_name)
            obj.write(f"Data/Individual_studies_final/2k/hlcamethod_cellranger_healthy2k/{variable_name}_2k_d.h5ad")


17
Healthy Mouse_Covid
Healthy Mouse_Copd_Covid
Healthy Mouse_Copd
Healthy Mouse_Age
Healthy Mouse_COPD_SHAM_SARSCov2
Healthy Mouse_Herpesvirus
Healthy Mouse_Hyperoxia
Healthy Mouse_TB
Healthy Mouse_Cancer
Healthy Mouse_Hypoxia
Healthy Mouse_Radiation
Healthy Mouse_Post_Sendai
Healthy Mouse_TumorKO
Healthy Mouse_fibrosis
Healthy Mouse_asthma
Healthy Mouse_Fibro_Age
Healthy Mouse_influenza
17
Disease Mouse_Covid
Disease Mouse_Copd_Covid
Disease Mouse_Copd
Disease Mouse_Age
Disease Mouse_COPD_SHAM_SARSCov2
Disease Mouse_Herpesvirus
Disease Mouse_Hyperoxia
Disease Mouse_TB
Disease Mouse_Cancer
Disease Mouse_Hypoxia
Disease Mouse_Radiation
Disease Mouse_Post_Sendai
Disease Mouse_TumorKO
Disease Mouse_fibrosis
Disease Mouse_asthma
Disease Mouse_Fibro_Age
Disease Mouse_influenza


Saving each set independantly as their healthy and disease variant for later integration